1) Enter your zip in cell 4
2) Retrieves lat, long from zip_code_database
3) Predicts GHI from lat, long and returns r-squared value in cells 10 & 11

In [1]:
import pandas as pd
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [3]:
zips=pd.read_csv('zip_code_database.csv')
zips

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42719,99926,PO BOX,0,Metlakatla,NaN,NaN,AK,Prince of Wales-Outer Ketchikan Borough,America/Metlakatla,907,NaN,US,55.14,-131.49,1140
42720,99927,PO BOX,0,Point Baker,NaN,NaN,AK,Prince of Wales-Hyder Census Area,America/Sitka,907,NaN,US,56.30,-133.57,48
42721,99928,PO BOX,0,Ward Cove,NaN,NaN,AK,Ketchikan Gateway Borough,America/Sitka,907,NaN,US,55.45,-131.79,1530
42722,99929,PO BOX,0,Wrangell,NaN,NaN,AK,Wrangell City and Borough,America/Sitka,907,NaN,US,56.41,-131.61,2145


In [4]:
# Take user ZIP input with error handling
user_input=input("Enter your ZIP: \n")
print(f'You entered {user_input}')
coords=zips.loc[zips['zip']==int(user_input)][['latitude','longitude']].values
while len(coords)==0:
    print(f'ERROR invalid ZIP: {user_input}')
    user_input=input("Re-enter your ZIP: \n")
    print(f'You entered {user_input}')
    coords=zips.loc[zips['zip']==int(user_input)][['latitude','longitude']].values

Enter your ZIP: 
1
You entered 1
ERROR invalid ZIP: 1
Re-enter your ZIP: 
90007
You entered 90007


In [5]:
coords

array([[  34.02, -118.28]])

In [6]:
df=pd.read_csv('pv_open_2020.csv')
df

,sc_gid,capacity_factor,global_horizontal_irradiance,capacity_mw,area_sq_km,latitude,longitude,distance_to_transmission_km
0,0,0.139177,3.354821,591.452887,18.482903,48.994,-122.735,0.673354
1,1,0.137451,3.306946,596.647266,18.645227,49.022,-122.575,6.875389
2,2,0.136000,3.293000,2.851200,0.089100,49.049,-122.414,7.901461
3,3,0.141827,3.417382,2014.383178,62.949474,48.900,-122.688,2.910129
4,4,0.137829,3.327924,3557.400778,111.168774,48.927,-122.529,102.926423
...,...,...,...,...,...,...,...,...
55514,55530,0.206913,5.196184,965.520000,30.172500,25.982,-97.349,22.586464
55515,55531,0.205765,5.174338,238.679140,7.458723,25.983,-97.231,9.374740
55516,55532,0.208000,5.208001,25.093151,0.784161,25.877,-97.583,4.844146
55517,55533,0.207652,5.203368,330.091200,10.315350,25.879,-97.465,5.204726


In [7]:
X = df[['latitude','longitude']].values
y = df['global_horizontal_irradiance'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)
regressor = RandomForestRegressor(n_estimators=10)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [9]:
pickle.dump(regressor, open('model.pkl','wb'))
model = pickle.load( open('model.pkl','rb'))

In [10]:
print(model.predict(coords))

[5.43178067]


In [11]:
r2_score(y_test, y_pred)

0.9980065855693727